#### Data Loading

In [ ]:
import datetime
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
sort_folder = './sort'
file_path_order = os.path.join(sort_folder, 'REP_ORD_ORDER.parquet')
file_path_job_code = os.path.join(sort_folder, 'REP_ORD_JOB_CODE.parquet')
file_path_order_state = os.path.join(sort_folder, 'REP_ORD_ORDER_STATE.parquet')
file_path_assignment = os.path.join(sort_folder, 'REP_ASN_ASSIGNMENT.parquet')
file_path_lab_resource = os.path.join(sort_folder, 'REP_LAB_RESOURCE.parquet')
file_path_lab_user = os.path.join(sort_folder, 'REP_LAB_USER.parquet')

# load tables with just the columns that you need
OO = pd.read_parquet(file_path_order,engine = 'pyarrow',columns = ['ORDER_ID','ORDER_NUM','JOB_CODE','ELIGIBLE'])
JC = pd.read_parquet(file_path_job_code,engine = 'pyarrow',columns = ['JOB_CODE_ID','NAME','CORE_DESCRIPTION'])
OOS = pd.read_parquet(file_path_order_state,engine = 'pyarrow',columns=[
    'ORDER_STATE_ID','FOR_ORDER','ORDER_NUM','LATEST_ASSIGNMENT','TOTAL_TIME_EN_ROUTE','TOTAL_TIME_ON_SITE', 'ONSITE_AT', 'COMPLETED', 'ENROUTE_AT'
])
AA = pd.read_parquet(file_path_assignment,engine = 'pyarrow',columns = ['ASSIGNMENT_ID','FOR_RESOURCE'])
LR = pd.read_parquet(file_path_lab_resource,engine = 'pyarrow',columns = ['RESOURCE_ID','FOR_USER'])
LU = pd.read_parquet(file_path_lab_user,engine = 'pyarrow',columns = ['USER_ID','LOGON_ID'])

In [ ]:
# merge the tables into a single dataset that can be analyzed
DF = OO.merge(JC,left_on = 'JOB_CODE',right_on='JOB_CODE_ID')
DF = DF.merge(OOS, left_on = 'ORDER_ID', right_on = 'FOR_ORDER')
DF = DF.merge(AA,left_on = 'LATEST_ASSIGNMENT',right_on = 'ASSIGNMENT_ID')
DF = DF.merge(LR,left_on = 'FOR_RESOURCE',right_on = 'RESOURCE_ID')
DF = DF.merge(LU,left_on = 'FOR_USER',right_on = 'USER_ID')

#### Data Type Conversion

In [ ]:
# Convert columns to appropriate data types
DF = DF.copy()

# Convert time columns to numeric
time_columns = ['TOTAL_TIME_EN_ROUTE', 'TOTAL_TIME_ON_SITE']
for col in time_columns:
    DF[col] = pd.to_numeric(DF[col], errors='coerce')

timestamp_col = ['ELIGIBLE','ONSITE_AT', 'COMPLETED', 'ENROUTE_AT']

for col in timestamp_col:
    DF[col] = pd.to_datetime(DF[col], errors='coerce')  

# Display the updated data types
print("Updated data types:")
print(DF.dtypes)

#### Missingness Assessment

In [ ]:
DF.isna().sum()

#### New Metric Calculation (Utilization Rate and Idle Time)

##### Daily Utilization Calculation

In [ ]:
# Calculate actual work duration using timestamps
DF['WORK_START'] = DF['ENROUTE_AT']  # When work actually started
DF['WORK_END'] = DF['COMPLETED']     # When work actually ended
DF['ONSITE_START'] = DF['ONSITE_AT'] # When onsite work started

# Calculate actual work duration in minutes
DF['ACTUAL_WORK_DURATION'] = (DF['WORK_END'] - DF['WORK_START']).dt.total_seconds() / 60

# Create date ranges for orders spanning multiple days
def get_work_dates(start_time, end_time):
    """Get all dates between start and end of work"""
    if pd.isna(start_time) or pd.isna(end_time):
        return []
    
    start_date = start_time.date()
    end_date = end_time.date()
    
    dates = []
    current_date = start_date
    while current_date <= end_date:
        dates.append(current_date)
        current_date += pd.Timedelta(days=1)
    
    return dates

# Calculate daily work time for each order
expanded_records = []

for idx, row in DF.iterrows():
    if pd.notna(row['WORK_START']) and pd.notna(row['WORK_END']):
        work_dates = get_work_dates(row['WORK_START'], row['WORK_END'])
        
        if len(work_dates) == 1:
            # Single day work
            expanded_records.append({
                'TECH_ID': row['LOGON_ID'],
                'DATE': work_dates[0],
                'DAILY_WORK_TIME': row['ACTUAL_WORK_DURATION'],
                'ORDER_ID': row['ORDER_ID'],
                'JOB_CODE': row['NAME'],
                'WORK_START': row['WORK_START'],
                'WORK_END': row['WORK_END']
            })
        else:
            # Multi-day work - distribute time across days
            total_duration = row['ACTUAL_WORK_DURATION']
            daily_duration = total_duration / len(work_dates)
            
            for work_date in work_dates:
                expanded_records.append({
                    'TECH_ID': row['LOGON_ID'],
                    'DATE': work_date,
                    'DAILY_WORK_TIME': daily_duration,
                    'ORDER_ID': row['ORDER_ID'],
                    'JOB_CODE': row['NAME'],
                    'WORK_START': row['WORK_START'],
                    'WORK_END': row['WORK_END']
                })

expanded_df = pd.DataFrame(expanded_records)

# Aggregate by (tech, date)
daily_utilization = expanded_df.groupby(['TECH_ID', 'DATE']).agg({
    'DAILY_WORK_TIME': 'sum',
    'ORDER_ID': 'count'
}).reset_index()

daily_utilization.rename(columns={
    'DAILY_WORK_TIME': 'TOTAL_MINUTES_WORKED',
    'ORDER_ID': 'JOBS_COUNT'
}, inplace=True)

# It is okay to assume 7 hours shift given mentor's domain knowledge
WORK_DAY_MINUTES = 7 * 60  # 420 minutes
daily_utilization['UTILIZATION_RATE'] = (
    daily_utilization['TOTAL_MINUTES_WORKED'] / WORK_DAY_MINUTES
).round(4)

In [ ]:
def utilization_distribution_plot(daily_utilization):
    """
    Create utilization distribution analysis
    """
    
    # Calculate key statistics
    mean_util = daily_utilization['UTILIZATION_RATE'].mean()
    
    # Create the histogram
    fig = go.Figure()

    # Add optimization area (shaded region from 0 to 100%)
    fig.add_vrect(
        x0=0, x1=1.0,
        fillcolor="rgba(255, 193, 7, 0.15)",
        layer="below",
        line_width=0,
        annotation_text="OPTIMIZATION ZONE",
        annotation_position="top",
        annotation=dict(
            font=dict(size=12, color="#f39c12", family="Arial Bold"),
            bgcolor="rgba(255, 255, 255, 0.8)",
            bordercolor="#f39c12",
            borderwidth=1
        )
    )
    
    # Add histogram
    fig.add_trace(go.Histogram(
        x=daily_utilization['UTILIZATION_RATE'],
        nbinsx=100,
        name='Daily Utilization',
        marker=dict(
            color='rgba(52, 152, 219, 0.7)',
            line=dict(color='rgba(52, 152, 219, 1.0)', width=1)
        ),
        opacity=0.8
    ))

    # Add 100% target line
    fig.add_vline(
        x=1.0, 
        line_dash="solid", 
        line_color="#27ae60",
        line_width=4,
        annotation_text="100% Utilization",
        annotation_position="top right",
        annotation=dict(
            font=dict(size=12, color="#27ae60", family="Arial Bold"),
            bgcolor="rgba(255, 255, 255, 0.9)",
            bordercolor="#27ae60",
            borderwidth=2
        )
    )
    
    # Add mean line
    fig.add_vline(
        x=mean_util, 
        line_dash="dash", 
        line_color="red",
        line_width=3,
        annotation_text=f"Current Average: {mean_util:.1%}",
        annotation_position="top"
    )

    # Update layout for clean storytelling
    fig.update_layout(
        title=dict(
            text=f'<b>Distribution of Daily Utilization Rate for Field Technician</b><br>' +
                 f'<span style="font-size:14px; color:#7f8c8d;">',
            font=dict(size=20, color='#2c3e50'),
            x=0.5,
            pad=dict(b=20)
        ),
        xaxis=dict(
            title="<b>Daily Utilization Rate</b>",
            tickformat='.0%',
            showgrid=True,
            gridcolor='rgba(128,128,128,0.2)'
        ),
        yaxis=dict(
            title="<b>Frequency</b>",
            showgrid=True,
            gridcolor='rgba(128,128,128,0.2)'
        ),
        width=1000,
        height=600,
        plot_bgcolor='white',
        showlegend=False,
        font=dict(family="Arial, sans-serif")
    )
    
    
    return fig

# Create the simplified story visualization
simple_fig = utilization_distribution_plot(daily_utilization)
simple_fig.show()

#### Hourly Utilization Calculation

In [ ]:
# Extract hourly data from work timestamps
expanded_df['WORK_START_HOUR'] = pd.to_datetime(expanded_df['WORK_START']).dt.hour
expanded_df['WORK_END_HOUR'] = pd.to_datetime(expanded_df['WORK_END']).dt.hour

# Hourly analysis with utilization and idle time calculations
def create_hourly_activity_data():
    hourly_records = []
    
    for idx, row in expanded_df.iterrows():
        if pd.notna(row['WORK_START_HOUR']) and pd.notna(row['WORK_END_HOUR']):
            start_hour = int(row['WORK_START_HOUR'])
            end_hour = int(row['WORK_END_HOUR'])
            duration = row['DAILY_WORK_TIME']
            
            # If work spans multiple hours, distribute time across hours
            if start_hour == end_hour:
                # Work completed within same hour
                hourly_records.append({
                    'TECH_ID': row['TECH_ID'],
                    'DATE': row['DATE'],
                    'HOUR': start_hour,
                    'WORK_MINUTES': duration,
                    'ORDER_COUNT': 1
                })
            else:
                # Work spans multiple hours - distribute evenly
                if end_hour < start_hour:  # Work crosses midnight
                    hours_worked = (24 - start_hour) + end_hour
                else:
                    hours_worked = end_hour - start_hour + 1
                
                minutes_per_hour = duration / hours_worked
                
                current_hour = start_hour
                while True:
                    hourly_records.append({
                        'TECH_ID': row['TECH_ID'],
                        'DATE': row['DATE'],
                        'HOUR': current_hour,
                        'WORK_MINUTES': minutes_per_hour,
                        'ORDER_COUNT': 1/hours_worked  # Fraction of order per hour
                    })
                    
                    current_hour = (current_hour + 1) % 24
                    if current_hour == (end_hour % 24):
                        break
    
    return pd.DataFrame(hourly_records)

hourly_df = create_hourly_activity_data()

# Calculate utilization and idle time for each tech-hour combination
HOUR_MINUTES = 60  # 60 minutes per hour

# Calculate utilization rate and idle time for each record
hourly_df['UTILIZATION_RATE'] = (hourly_df['WORK_MINUTES'] / HOUR_MINUTES).clip(upper=1.0)  # Cap at 100%
hourly_df['IDLE_MINUTES'] = HOUR_MINUTES - hourly_df['WORK_MINUTES']
hourly_df['IDLE_MINUTES'] = hourly_df['IDLE_MINUTES'].clip(lower=0)  # No negative idle time
hourly_df['IDLE_TIME_PCT'] = (hourly_df['IDLE_MINUTES'] / HOUR_MINUTES) * 100

# Add additional time dimensions for analysis
hourly_df['DAY_OF_WEEK'] = pd.to_datetime(hourly_df['DATE']).dt.day_name()
hourly_df['MONTH'] = pd.to_datetime(hourly_df['DATE']).dt.month_name()
hourly_df['IS_WEEKEND'] = pd.to_datetime(hourly_df['DATE']).dt.dayofweek.isin([5, 6])
hourly_df['IS_BUSINESS_HOUR'] = hourly_df['HOUR'].between(8, 17)  # 8 AM to 5 PM


In [ ]:
# hourly summary from hourly_df
hourly_summary = hourly_df.groupby('HOUR').agg({
    'WORK_MINUTES': 'sum',
    'TECH_ID': 'nunique',
    'ORDER_COUNT': 'sum',
    'UTILIZATION_RATE': 'mean',
    'IDLE_MINUTES': 'sum',
    'IDLE_TIME_PCT': 'mean'
}).reset_index()

hourly_summary.rename(columns={
    'WORK_MINUTES': 'TOTAL_WORK_MINUTES',
    'TECH_ID': 'UNIQUE_TECHS',
    'ORDER_COUNT': 'TOTAL_ORDERS',
    'UTILIZATION_RATE': 'AVG_UTILIZATION_RATE',
    'IDLE_MINUTES': 'TOTAL_IDLE_MINUTES',
    'IDLE_TIME_PCT': 'AVG_IDLE_TIME_PCT'
}, inplace=True)

def create_utilization_hourly_plot(hourly_summary):
    """
    Create a utilization/idle time hourly analysis
    """
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            'Idle Time Pattern and Order Volume by Hour', 
            'Utilization Rate Throughout the Day'
        ),
        specs=[[{"secondary_y": True}], [{"secondary_y": False}]],
        vertical_spacing=0.15
    )
    
    # TOP PLOT: Idle Time and Order Count
    fig.add_trace(
        go.Scatter(
            x=hourly_summary['HOUR'], 
            y=hourly_summary['AVG_IDLE_TIME_PCT'],
            mode='lines+markers',
            name='Idle Time %',
            line=dict(color='#e74c3c', width=4),
            marker=dict(size=10, color='#e74c3c'),
            hovertemplate='<b>Hour %{x}:00</b><br>' +
                         'Idle Time: %{y:.1f}%<br>' +
                         '<extra></extra>'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=[None], y=[None],
            mode='markers',
            name='Business Hours (8-17)',
            marker=dict(size=10, color='rgba(39,174,96,0.3)'),
            showlegend=True
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=hourly_summary['HOUR'],
            y=hourly_summary['TOTAL_ORDERS'],
            name='Order Count',
            marker=dict(color='rgba(52, 152, 219, 0.3)', line=dict(color='#3498db', width=1)),
            hovertemplate='<b>Hour %{x}:00</b><br>' +
                         'Orders: %{y:.1f}<br>' +
                         '<extra></extra>',
            yaxis='y2'
        ),
        row=1, col=1, secondary_y=True
    )
    
    # BOTTOM PLOT: Utilization Rate 
    fig.add_trace(
        go.Scatter(
            x=hourly_summary['HOUR'],
            y=hourly_summary['AVG_UTILIZATION_RATE'] * 100,  # Convert to percentage
            mode='lines+markers',
            name='Utilization Rate %',
            line=dict(color='#27ae60', width=4),
            marker=dict(size=10, color='#27ae60'),
            hovertemplate='<b>Hour %{x}:00</b><br>' +
                         'Utilization: %{y:.1f}%<br>' +
                         '<extra></extra>'
        ),
        row=2, col=1
    )
    
    highest_idle_hour = int(hourly_summary.loc[hourly_summary['AVG_IDLE_TIME_PCT'].idxmax(), 'HOUR'])
    lowest_idle_hour = int(hourly_summary.loc[hourly_summary['AVG_IDLE_TIME_PCT'].idxmin(), 'HOUR'])
    peak_utilization_hour = int(hourly_summary.loc[hourly_summary['AVG_UTILIZATION_RATE'].idxmax(), 'HOUR'])
    
    highest_idle_value = hourly_summary['AVG_IDLE_TIME_PCT'].max()
    lowest_idle_value = hourly_summary['AVG_IDLE_TIME_PCT'].min()
    peak_utilization_value = hourly_summary['AVG_UTILIZATION_RATE'].max() * 100
    
    # Highest idle time annotation
    fig.add_annotation(
        x=highest_idle_hour, y=highest_idle_value,
        text=f"{highest_idle_hour:02d}:00<br>{highest_idle_value:.1f}% idle",
        showarrow=True,
        arrowcolor="#e74c3c",
        bgcolor="rgba(231,76,60,0.9)",
        bordercolor="#e74c3c",
        font=dict(color="white", size=11, family="Arial Black"),
        ax=0, ay=-50,
        row=1, col=1
    )
    
    # Lowest idle time annotation
    fig.add_annotation(
        x=lowest_idle_hour, y=lowest_idle_value,
        text=f"{lowest_idle_hour:02d}:00<br>{lowest_idle_value:.1f}% idle",
        showarrow=True,
        arrowcolor="#27ae60",
        bgcolor="rgba(39,174,96,0.9)",
        bordercolor="#27ae60",
        font=dict(color="white", size=11, family="Arial Black"),
        ax=0, ay=50,
        row=1, col=1
    )
    
    # Peak utilization annotation
    fig.add_annotation(
        x=peak_utilization_hour, y=peak_utilization_value,
        text=f"{peak_utilization_hour:02d}:00<br>{peak_utilization_value:.1f}% utilized",
        showarrow=True,
        arrowcolor="#27ae60",
        bgcolor="rgba(39,174,96,0.9)",
        bordercolor="#27ae60",
        font=dict(color="white", size=11, family="Arial Black"),
        ax=0, ay=-50,
        row=2, col=1
    )
    
    fig.update_layout(
        title=dict(
            text=f'<b>Field Technician Hourly Analysis: Idle Time vs Utilization Efficiency</b><br>' +
                 f'<span style="font-size:14px; color:#e67e22;">',
            font=dict(size=18, color='#2c3e50'),
            x=0.5
        ),
        height=800,
        width=1200,
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom", y=1.02,
            xanchor="center", x=0.5
        )
    )
    
    fig.update_xaxes(title_text="<b>Hour of Day</b>", row=2, col=1)
    fig.update_yaxes(title_text="<b>Idle Time %</b>", row=1, col=1)
    fig.update_yaxes(title_text="<b>Order Count</b>", row=1, col=1, secondary_y=True)
    fig.update_yaxes(title_text="<b>Utilization Rate %</b>", row=2, col=1)
    
    fig.add_hline(y=100, line_dash="dash", line_color="red", 
                  annotation_text="100% Utilization Target", row=2, col=1)
    
    fig.add_vrect(x0=8, x1=17, fillcolor="rgba(39,174,96,0.1)", layer="below", line_width=0, row=1, col=1)
    fig.add_vrect(x0=8, x1=17, fillcolor="rgba(39,174,96,0.1)", layer="below", line_width=0, row=2, col=1)
    
    return fig


story_fig = create_utilization_hourly_plot(hourly_summary, hourly_df)
story_fig.show()